In [ ]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import shotchartdetail
import pandas as pd
from datetime import datetime
import json
from nba_api.stats.endpoints import teamyearbyyearstats
import time
import pickle

In [ ]:
players_l = [
    "Cade Cunningham",
    "Tyler Herro",
    "Evan Mobley",
    "Alperen Sengun",
    "Victor Wembanyama",
    "Jalen Williams",
    "Jalen Brunson",
    "Paolo Banchero",
    "Scottie Barnes",
    "Tyrese Maxey",
    "Jaren Jackson Jr.",
    "Anthony Edwards",
    "Shai Gilgeous-Alexander",
    "Tyrese Haliburton",
    "De'Aaron Fox",
    "Lauri Markkanen",
    "Darius Garland",
    "Ja Morant",
    "Jarrett Allen",
    "LaMelo Ball",
    "Dejounte Murray",
    "Fred VanVleet",
    "Andrew Wiggins",
    "Jaylen Brown",
    "Julius Randle",
    "Zion Williamson",
    "Zach LaVine",
    "Mike Conley",
    "Pascal Siakam",
    "Bam Adebayo",
    "Trae Young",
    "Donovan Mitchell",
    "Jayson Tatum",
    "Luka Doncic",
    "Devin Booker",
    "Rudy Gobert",
    "Domantas Sabonis",
    "Brandon Ingram",
    "Nikola Vucevic",
    "Nikola Jokic",
    "Ben Simmons",
    "Khris Middleton",
    "D'Angelo Russell",
    "Joel Embiid",
    "Karl-Anthony Towns",
    "Bradley Beal",
    "Victor Oladipo",
    "Goran Dragic",
    "Kristaps Porzingis",
    "Giannis Antetokounmpo",
    "Kemba Walker",
    "Gordon Hayward",
    "DeAndre Jordan",
    "Andre Drummond",
    "Draymond Green",
    "Kawhi Leonard",
    "Isaiah Thomas",
    "Kyle Lowry",
    "Klay Thompson",
    "Jimmy Butler",
    "DeMarcus Cousins",
    "Kyle Korver",
    "Jeff Teague",
    "DeMar DeRozan",
    "Anthony Davis",
    "Stephen Curry",
    "John Wall",
    "Paul Millsap",
    "Damian Lillard",
    "Jrue Holiday",
    "James Harden",
    "Kyrie Irving",
    "Paul George",
    "Joakim Noah",
    "Tyson Chandler",
    "Brook Lopez",
    "Marc Gasol",
    "Roy Hibbert",
    "LaMarcus Aldridge",
    "Luol Deng",
    "Andrew Bynum",
    "Andre Iguodala",
    "Blake Griffin",
    "Russell Westbrook",
    "Kevin Love",
    "David Lee",
    "Zach Randolph",
    "Kevin Durant",
    "Rajon Rondo",
    "Derrick Rose",
    "Deron Williams",
    "Al Horford",
    "Chris Kaman",
    "Gerald Wallace",
    "Danny Granger",
    "Devin Harris",
    "Jameer Nelson",
    "Mo Williams",
    "Chris Paul",
    "Brandon Roy",
    "David West",
    "Dwight Howard",
    "Joe Johnson",
    "Carmelo Anthony",
    "Carlos Boozer",
    "Caron Butler",
    "Josh Howard",
    "Mehmet Okur",
    "Pau Gasol",
    "Chris Bosh",
    "Chauncey Billups",
    "Richard Hamilton",
    "Tony Parker",
    "Manu Ginobili",
    "Rashard Lewis",
    "Antawn Jamison",
    "Amar'e Stoudemire",
    "LeBron James",
    "Dwyane Wade",
    "Gilbert Arenas"
]

In [ ]:
len(players_l)

In [ ]:
injuries = pd.read_csv('injuries.csv')

In [ ]:
new = []
special = []
inj = {}
temp = []
for index, row in injuries.iterrows():
    name = ''
    temp.append(str(row['Acquired'])[1:])
    if ('out for season' in str(row['Notes']) or 'out indefinitely' in str(row['Notes'])) and (datetime.strptime(row['Date'], '%Y-%m-%d').month > 9 or datetime.strptime(row['Date'], '%Y-%m-%d').month < 4):
        if datetime.strptime(row['Date'], '%Y-%m-%d').month > 6:
            season = int(datetime.strptime(row['Date'], '%Y-%m-%d').year)+1
        else:
            season = int(datetime.strptime(row['Date'], '%Y-%m-%d').year)
        if str(row['Acquired']) == 'nan':
            name = str(row['Relinquished']).strip()
        else:
            name = str(row['Relinquished']).strip()
        if name in players_l:
            if name not in inj:
                inj[name] = []
            inj[name].append([season, str(row['Notes'])])    
            new.append([season, str(row['Notes'])])
    #if str(row['Acquired'])[1:] in players:
    else:
        season = 0
        dist = datetime.strptime(row['Date'], '%Y-%m-%d') - datetime.strptime(injuries.iloc[index-1]['Date'], '%Y-%m-%d') 
        if datetime.strptime(row['Date'], '%Y-%m-%d').month > 6:
            season = int(datetime.strptime(row['Date'], '%Y-%m-%d').year)+1
        else:
            season = int(datetime.strptime(row['Date'], '%Y-%m-%d').year)
        if dist.days > 30:
            if name in players_l:
                if name not in inj:
                    inj[name] = []
                inj[name].append([season, str(row['Notes'])])    
                new.append([season, str(row['Notes'])])

In [ ]:
team_abbr = ['PHI',
'MIL',	
'CHI',	
'CLE',	
'BOS',	
'LAC',	
'MEM',	
'ATL',	
'MIA',	
'CHA',	
'UTA',	
'SAC',	
'NYK',	
'LAL',	
'ORL',	
'DAL',	
'BKN',	
'DEN',	
'IND',	
'NOP',	
'DET',	
'TOR',	
'HOU',	
'SAS',	
'PHX',	
'OKC',	
'MIN',	
'POR',	
'GSW',	
'WAS',
'CHO',
'NOH',
'SEA',
'NJN'
]

In [ ]:
def playerInfo(id, name):
    pdata = []
    c = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(id)
    d = json.loads(c.get_json())

    team_map = {}
    for i in range(len(d['resultSets'][1]['rowSet'])):
        if '   ' not in d['resultSets'][1]['rowSet'][i][1]: 
            if i < len(d['resultSets'][1]['rowSet'])-2:
                if d['resultSets'][1]['rowSet'][i][1] == d['resultSets'][1]['rowSet'][i+1][1] and d['resultSets'][1]['rowSet'][i+1][1] == d['resultSets'][1]['rowSet'][i+2][1]:
                    team_map[int(d['resultSets'][1]['rowSet'][i][1][0:2]+d['resultSets'][1]['rowSet'][i][1][5:])] = [d['resultSets'][1]['rowSet'][i+1][2], d['resultSets'][1]['rowSet'][i+2][2]]
            if int(d['resultSets'][1]['rowSet'][i][1][0:2]+d['resultSets'][1]['rowSet'][i][1][5:]) not in team_map:
                team_map[int(d['resultSets'][1]['rowSet'][i][1][0:2]+d['resultSets'][1]['rowSet'][i][1][5:])] = d['resultSets'][1]['rowSet'][i][2]
        time.sleep(0.1)
    info = commonplayerinfo.CommonPlayerInfo(id)
    # getting total player data (height, weight, date of birth, position) 
    data = json.loads(info.get_json())
    date = data['resultSets'][0]['rowSet'][0][7]
    h = data['resultSets'][0]['rowSet'][0][11]
    hs = h.split('-')
    height = int(hs[0])*12 + int(hs[1]) 
    w = int(data['resultSets'][0]['rowSet'][0][12])
    position = data['resultSets'][0]['rowSet'][0][15]
    
    u_factors = []
    for i in d['resultSets'][1]['rowSet']:
        if i[3] in team_abbr: 
            time.sleep(0.2)
            shotdata = [0, 0, 0, 0]
            if '   ' in i[1]:
                break
            if type(team_map[int(i[1][0:2]+i[1][5:])]) == int:
                s = shotchartdetail.ShotChartDetail(team_id=team_map[int(i[1][0:2]+i[1][5:])], player_id=id, season_nullable=i[1], context_measure_simple = 'FGA')
                # shot data
                shots = json.loads(s.get_json())
                for j in shots['resultSets'][0]['rowSet']:
                    if j[15] == '24+ ft.':
                        shotdata[3] += 1
                    if j[15] == '16-24 ft.':
                        shotdata[2] += 1
                    if j[15] == '8-16 ft.':
                        shotdata[1] += 1
                    if j[15] == 'Less Than 8 ft.':
                        shotdata[0] += 1
                # usage factor
                u = teamyearbyyearstats.TeamYearByYearStats(team_map[int(i[1][0:2]+i[1][5:])]).get_json()
                us = json.loads(u)
                for j in us['resultSets'][0]['rowSet']:
                    if j[3] == i[1] and shotdata != [0, 0, 0, 0]:
                        u_factors.append((48) / ((j[16]/j[4]) + 0.44*(j[22]/82) + (j[30]/82)))
            else:
                for l in team_map[int(i[1][0:2]+i[1][5:])]:
                    s = shotchartdetail.ShotChartDetail(team_id=l, player_id=id, season_nullable=i[1], context_measure_simple = 'FGA')
                    shots = json.loads(s.get_json())
                    for j in shots['resultSets'][0]['rowSet']:
                        if j[15] == '24+ ft.':
                            shotdata[3] += 1
                        if j[15] == '16-24 ft.':
                            shotdata[2] += 1
                        if j[15] == '8-16 ft.':
                            shotdata[1] += 1
                        if j[15] == 'Less Than 8 ft.':
                            shotdata[0] += 1
                    if l != -1:
                        u = teamyearbyyearstats.TeamYearByYearStats(l).get_json()
                        us = json.loads(u)
                        for j in us['resultSets'][0]['rowSet']:
                            if j[3] == i[1] and shotdata != [0, 0, 0, 0]:
                                u_factors.append((48) / ((j[16]/j[4]) + 0.44*(j[22]/82) + (j[30]/82)))
            if len(pdata) == 0:
                n = False
                for t_t in i:
                    if t_t == None or str(t_t) == "None":
                        n = True
                if n == True:
                    break
                temp = [height, w, position]
                # season
                temp.append(i[1][0:2]+i[1][5:])
                # age
                temp.append((datetime.strptime(i[1][0:2]+i[1][5:], '%Y') - datetime.strptime(date[:10], '%Y-%M-%d')).days//365)
                # team id
                temp.append(i[2])
                # games played
                temp.append(i[5])
                # minutes played
                temp.append(round(i[9]/i[5], 2))
                # fgm
                temp.append(round(i[10]/i[5], 2))
                # fga
                temp.append(round(i[11]/i[5], 2))
                # 3fgm
                temp.append(round(i[13]/i[5], 2))
                # 3fga
                temp.append(round(i[14]/i[5], 2))
                # reb
                temp.append(round(i[21]/i[5], 2))
                # ast
                temp.append(round(i[22]/i[5], 2))
                # tov
                temp.append(round(i[23]/i[5], 2))
                # stl
                temp.append(round(i[24]/i[5], 2))
                # blk
                temp.append(round(i[25]/i[5], 2))
                # opponents blocking their shot (blka)
                temp.append(round(i[26]/i[5], 2))
                # pf
                temp.append(round(i[27]/i[5], 2))
                # opponents fouling them (pfd)
                temp.append(round(i[28]/i[5], 2))
                # pts
                temp.append(round(i[29]/i[5], 2))
            
                if (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]) > 0:
                    # less than 8 feet
                    temp.append(round(shotdata[0] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                    # 8-16 feet
                    temp.append(round(shotdata[1] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                    # 16-24 feet
                    temp.append(round(shotdata[2] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                    # 24+ feet
                    temp.append(round(shotdata[3] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                else:
                    temp.append(0)
                    temp.append(0)
                    temp.append(0)
                    temp.append(0)
    
                if i[2] == -1:
                    usg_player = ((round(i[11]/i[5], 2))+0.44*(round(i[16]/i[5], 2))+(round(i[22]/i[5], 2)))/(round(i[9]/i[5], 2))
                    if len(u_factors) > 1:
                        temp.append(100*usg_player*(u_factors[0] + u_factors[1])/2)
                        u_factors = u_factors[2:]
                    else:
                        temp.append(100*usg_player*(u_factors[0]))
                        u_factors = u_factors[1:]
                else:
                    usg_player = ((round(i[11]/i[5], 2))+0.44*(round(i[16]/i[5], 2))+(round(i[22]/i[5], 2)))/(round(i[9]/i[5], 2))
                    temp.append(100*usg_player*u_factors[0])
                    u_factors = u_factors[1:]
                
                injured = 0
                if name in list(inj.keys()):
                    for p in inj[name]:
                        if p[0] == i[1][0:2]+i[1][5:]:
                            injured = 1
                temp.append(injured)
                
                pdata.append(temp)
            elif (i[1][0:2]+i[1][5:]) != pdata[-1][3]:
                n = False
                for t_t in i:
                    if t_t == None or str(t_t) == "None":
                        n = True
                if n == True:
                    break
                temp = [height, w, position]
                # season
                temp.append(i[1][0:2]+i[1][5:])
                # age
                temp.append((datetime.strptime(i[1][0:2]+i[1][5:], '%Y') - datetime.strptime(date[:10], '%Y-%M-%d')).days//365)
                # team id
                temp.append(i[2])
                # games played
                temp.append(i[5])
                # minutes played
                temp.append(round(i[9]/i[5], 2))
                # fgm
                temp.append(round(i[10]/i[5], 2))
                # fga
                temp.append(round(i[11]/i[5], 2))
                # 3fgm
                temp.append(round(i[13]/i[5], 2))
                # 3fga
                temp.append(round(i[14]/i[5], 2))
                # reb
                temp.append(round(i[21]/i[5], 2))
                # ast
                temp.append(round(i[22]/i[5], 2))
                # tov
                temp.append(round(i[23]/i[5], 2))
                # stl
                temp.append(round(i[24]/i[5], 2))
                # blk
                temp.append(round(i[25]/i[5], 2))
                # opponents blocking their shot (blka)
                temp.append(round(i[26]/i[5], 2))
                # pf
                temp.append(round(i[27]/i[5], 2))
                # opponents fouling them (pfd)
                temp.append(round(i[28]/i[5], 2))
                # pts
                temp.append(round(i[29]/i[5], 2))
            
                if (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]) > 0:
                    # less than 8 feet
                    temp.append(round(shotdata[0] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                    # 8-16 feet
                    temp.append(round(shotdata[1] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                    # 16-24 feet
                    temp.append(round(shotdata[2] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                    # 24+ feet
                    temp.append(round(shotdata[3] / (shotdata[0] + shotdata[1] + shotdata[2] + shotdata[3]), 2)*100)
                else:
                    temp.append(0)
                    temp.append(0)
                    temp.append(0)
                    temp.append(0)
    
                if i[2] == -1:
                    usg_player = ((round(i[11]/i[5], 2))+0.44*(round(i[16]/i[5], 2))+(round(i[22]/i[5], 2)))/(round(i[9]/i[5], 2))
                    if len(u_factors) > 1:
                        temp.append(100*usg_player*(u_factors[0] + u_factors[1])/2)
                        u_factors = u_factors[2:]
                    else:
                        temp.append(100*usg_player*(u_factors[0]))                
                        u_factors = u_factors[1:]
                else: 
                    usg_player = ((round(i[11]/i[5], 2))+0.44*(round(i[16]/i[5], 2))+(round(i[22]/i[5], 2)))/(round(i[9]/i[5], 2))
                    temp.append(100*usg_player*u_factors[0])
                    u_factors = u_factors[1:]
    
                injured = 0
                if name in list(inj.keys()):
                    for p in inj[name]:
                        if p[0] == int(i[1][0:2]+i[1][5:]):
                            injured = 1
                temp.append(injured)
                    
                pdata.append(temp)
            time.sleep(0.1)
    return pdata

In [ ]:
def data(players_l):
    out = []
    for name in players_l:
        print(name)
        a = players.find_players_by_full_name(str(name))
        
        result = playerInfo(a[0]['id'], str(name))
        for i in result:
            out.append(i)
    return out

In [ ]:
# run only if continuing from a previous run
import pickle
with open("dataset", "rb") as fp: 
   test = pickle.load(fp)

In [ ]:
test = []

In [ ]:
# using for loop so that once you get a timeout error, you can save it using pickle and then restart next time
for i in range(0, 120):
    test.append(data([players_l[i]]))

In [ ]:
p = []
for i in test:
    if i not in p:
        if type(i[0]) is not int:
            for j in i:
                p.append(j)
        else:
            p.append(i)

In [ ]:
with open("dataset", "wb") as fp:   
   pickle.dump(p, fp)